基于已评分的高分电影相似度做的协同推荐

导入movielen数据

In [1]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt

import random
from sklearn.model_selection import train_test_split

# Reading ratings file
# Ignore the timestamp column
ratings = pd.read_csv('ratings.csv', sep='\t', encoding='latin-1', usecols=['user_id', 'movie_id', 'rating'])

# Reading users file
users = pd.read_csv('users.csv', sep='\t', encoding='latin-1', usecols=['user_id', 'gender', 'zipcode', 'age_desc', 'occ_desc'])

# Reading movies file
movies = pd.read_csv('movies.csv', sep='\t', encoding='latin-1', usecols=['movie_id', 'title', 'genres'])

将movies.genres转为数组的格式便于后续用tfidf做单词的数字化。

从
0     Animation|Children's|Comedy
1    Adventure|Children's|Fantasy
2                  Comedy|Romance
3                    Comedy|Drama
4                          Comedy

到
0     [u'Animation', u"Children's", u'Comedy']
1    [u'Adventure', u"Children's", u'Fantasy']
2                      [u'Comedy', u'Romance']
3                        [u'Comedy', u'Drama']
4                                  [u'Comedy']

In [2]:
movies.genres.head()

0     Animation|Children's|Comedy
1    Adventure|Children's|Fantasy
2                  Comedy|Romance
3                    Comedy|Drama
4                          Comedy
Name: genres, dtype: object

In [3]:
# Break up the big genre string into a string array
movies['genres'] = movies['genres'].str.split('|')
# Convert genres to string value
movies['genres'] = movies['genres'].fillna("").astype('str')

movies.genres.head()

0     [u'Animation', u"Children's", u'Comedy']
1    [u'Adventure', u"Children's", u'Fantasy']
2                      [u'Comedy', u'Romance']
3                        [u'Comedy', u'Drama']
4                                  [u'Comedy']
Name: genres, dtype: object

关于tfidf的说明见，

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['genres'])
tfidf_matrix.shape

(3883, 127)

由于TfidfVectorizer已经将向量标准化为长度1，所以可以直接用两个向量的点积作为cosine的值来判断向量之间的相似程度；因此可以用性能更好的linear_kernel来处理。

In [5]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim[:4, :4]

array([[1.        , 0.14193614, 0.09010857, 0.1056164 ],
       [0.14193614, 1.        , 0.        , 0.        ],
       [0.09010857, 0.        , 1.        , 0.1719888 ],
       [0.1056164 , 0.        , 0.1719888 , 1.        ]])

准备索引，便于后续查找数据，

titles：根据索引得到电影名称
indices：根据电影名称得到索引
indices_movid：根据电影id得到电影的索引（电影id与索引的对应并不完全对齐）

In [6]:
# Build a 1-dimensional array with movie titles
titles = movies['title']
indices = pd.Series(movies.index, index=movies['title'])

# Another one for movie id
# movie_id -> movie index
indices_movid = pd.Series(movies.index, index=movies['movie_id'])

根据电影名称，返回movie标签信息最接近的20个电影并返回，简单验证推荐的功能；
cosine_sim[10]的数组，即movie索引为10的电影，与所有索引的电影的相似度数值；

In [7]:
# Function that get movie recommendations based on the cosine similarity score of movie genres
def genre_recommend_by_title(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[:20]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

print(genre_recommend_by_title('Good Will Hunting (1997)').head(20))

13                                         Nixon (1995)
25                                       Othello (1995)
26                                  Now and Then (1995)
29    Shanghai Triad (Yao a yao yao dao waipo qiao) ...
30                               Dangerous Minds (1995)
35                              Dead Man Walking (1995)
39                      Cry, the Beloved Country (1995)
42                                   Restoration (1995)
52                                      Lamerica (1994)
54                                       Georgia (1995)
56                         Home for the Holidays (1995)
61                            Mr. Holland's Opus (1995)
66                                      Two Bits (1995)
77                           Crossing Guard, The (1995)
79         White Balloon, The (Badkonake Sefid ) (1995)
81                      Antonia's Line (Antonia) (1995)
82      Once Upon a Time... When We Were Colored (1995)
89                   Journey of August King, The

将数据集以默认1：3的比例以随机的方式分为训练和验证的部分，后续用验证集来验证推荐效果。
具体见，
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
    

In [8]:
# split into train & test dataset
ratings_train, ratings_test = train_test_split(ratings)

In [9]:
def get_first_n(l, n=3):
    """Get the first n results from l
    l is like,
    l = [(1, 1.0), (2, 1.0), (3, 0.8), (4, 0.75), (5, 0.6), (6, 0.4), (7, 1.0), (8, 1.0)]
    
    randomnize and return n of them if candidates more than n
    otherwise just sort and return first n of them
    """
    l = sorted(l, key=lambda x:x[1], reverse=True)
    top = l[0][-1]
    
    result = []
    for x in l:
        if x[1] == top:
            result.append(x)
    
    if len(result) >= n:
        random.shuffle(result)
        return result[:n]
    
    else:
        return l[:n]

In [10]:
def genre_recommend_by_movid_list(movie_id_list, count=10):
    """
    Input: highest rated movies id
    Output: recommend movie id list limited by count
    """
    recommend_movie_index_list = []
    sim_scores_full = []
    
    # Convert from movie id list to movie index list
    idx_list = [indices_movid[movie_id] for movie_id in movie_id_list]
    for idx in idx_list:
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        sim_scores_full.extend(sim_scores)
        
    # Remove watched movies
    for sim_score in sim_scores_full:
        if sim_score[0] in movie_id_list:
            sim_scores_full.remove(sim_score)

    # Only return <count> results
    recommend_movie_index_list = get_first_n(sim_scores_full, n=count)
    recommend_movie_index_list = [x[0] for x in recommend_movie_index_list]
    
    #print(recommend_movie_index_list)
    # Convert from movie index list to movie id list
    recommend_movie_id_list = \
        [np.asscalar(movies[movies.index == idx].movie_id) for idx in recommend_movie_index_list]

    return recommend_movie_id_list

简单验证基于用户的高评分电影的相似电影做的推荐列表结果
考虑到实际情况，为了避免每次推荐都是同样的电影，对所有最高评分的相似的电影做了随机返回，因此每次推荐的结果都会不一样。

In [11]:
def print_recomm_result_by_ratings_data(ratings, rated_movie_limit=10):
    #for user_id in np.sort(ratings['user_id'].unique()):
    for user_id in np.arange(1, 10):
        highest_rate_movie_id = \
            ratings[(ratings.user_id == user_id) & \
                (ratings.rating == np.max(ratings[ratings.user_id == user_id].rating))][['movie_id']]
        
        highest_rate_movie_id_list = highest_rate_movie_id.movie_id.values
        # Performance will have significant impact if not limit the number of
        # rated movies for filtering 
        random.shuffle(highest_rate_movie_id_list)
        highest_rate_movie_id_list = highest_rate_movie_id_list[:rated_movie_limit]
        
        #print('debug high rate movie len:', len(highest_rate_movie_id_list))
        
        recommend_list = genre_recommend_by_movid_list(highest_rate_movie_id_list)
        print(user_id, recommend_list)
        
print_recomm_result_by_ratings_data(ratings)

(1, [1586, 3759, 3776, 3628, 631, 1032, 3062, 1489, 1024, 1242])
(2, [380, 1197, 2405, 2406, 14, 26, 27, 30, 31, 36])
(3, [380, 1197, 2405, 2406, 368, 1304, 1378, 1379, 1259, 5])
(4, [692, 748, 2600, 3527, 3643, 1222, 1523, 1037, 3062, 3654])
(5, [3326, 2408, 1965, 2427, 1270, 3033, 1242, 3753, 2028, 718])
(6, [1233, 3062, 3654, 1586, 3628, 3643, 110, 3753, 2028, 1242])
(7, [1037, 1586, 3654, 2028, 2571, 2722, 1222, 286, 3654, 1586])
(8, [3654, 2692, 748, 2699, 110, 2600, 172, 1242, 2427, 1037])
(9, [3753, 1222, 3377, 2972, 1696, 527, 3628, 3654, 336, 1619])


通过推荐列表的命中率验证效果，使用验证集来验证推荐结果，训练集用来生成推荐列表。
由于movielen里除了有评价过的电影还有相应的评分，为了贴合实际，认为，
只有命中且评分高于该用户的p80的评分（有些人习惯打高分，其他人反之），才算命中

会执行很久，但可以像下列例子那样仅验证头100个user的推荐的命中率

In [12]:
def hit_ratio_benchmark(ratings_train, ratings_test, rated_movie_limit=10):
    """
    for each user_id
    1. get recommend list using ratings_train rated movies
    2. use ratings_test rated movies to validate hit ratio
    it is considered hit when,
    1. user rated this movie
    2. the rate is >= this user's p80 rate in ratings_train
    """
    hit, count = (0., 0.)
    
    #for user_id in np.sort(ratings_test['user_id'].unique()):
    for user_id in np.arange(1, 100):
        #print('user_id %s' % user_id)
        highest_rate_movie_id = \
            ratings_train[(ratings_train.user_id == user_id) & \
                (ratings_train.rating == np.max(ratings_train[ratings_train.user_id == user_id].rating))][['movie_id']]
        
        highest_rate_movie_id_list = highest_rate_movie_id.movie_id.values
        # Performance will have significant impact if not limit the number of
        # rated movies for filtering 
        random.shuffle(highest_rate_movie_id_list)
        highest_rate_movie_id_list = highest_rate_movie_id_list[:rated_movie_limit]
        
        recommend_list = genre_recommend_by_movid_list(highest_rate_movie_id_list)
        #print(user_id, recommend_list)
        
        for item in recommend_list:
            count += 1
            x = ratings_test[ratings_test.user_id == user_id][['movie_id', 'rating']]
            if x[x.movie_id == item].empty:
                continue
            elif x[x.movie_id == item].rating.values < np.percentile(x.rating.values, 80):
                continue
            else:
                hit += 1
    
    print(hit, count)
    #hit_ratio = hit / ratings_test.movie_id.count() * 1.0
    hit_ratio = hit / count
    return hit_ratio
        
        
hit_ratio = hit_ratio_benchmark(ratings_train, ratings_test)
hit_ratio *= 100
print('hit ratio percentage: %.10f%%' % hit_ratio)

(19.0, 990.0)
hit ratio percentage: 1.9191919192%


NDCG原理见，
http://sofasofa.io/forum_main_post.php?postid=1002561

In [13]:
def dcg_at_k(r, k, method=0):
    """Score is discounted cumulative gain (dcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
    >>> r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
    >>> dcg_at_k(r, 1)
    3.0
    >>> dcg_at_k(r, 1, method=1)
    3.0
    >>> dcg_at_k(r, 2)
    5.0
    >>> dcg_at_k(r, 2, method=1)
    4.2618595071429155
    >>> dcg_at_k(r, 10)
    9.6051177391888114
    >>> dcg_at_k(r, 11)
    9.6051177391888114
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    Returns:
        Discounted cumulative gain
    """
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, k, method=0):
    """Score is normalized discounted cumulative gain (ndcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
    >>> r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
    >>> ndcg_at_k(r, 1)
    1.0
    >>> r = [2, 1, 2, 0]
    >>> ndcg_at_k(r, 4)
    0.9203032077642922
    >>> ndcg_at_k(r, 4, method=1)
    0.96519546960144276
    >>> ndcg_at_k([0], 1)
    0.0
    >>> ndcg_at_k([1], 2)
    1.0
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    Returns:
        Normalized discounted cumulative gain
    """
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

对所有验证集的用户的推荐列表，计算ndcg并计算他们的平均值，作为该推荐算法的ndcg分数
由于ndcg计算的是推荐列表的顺序的精确度，因此如果验证集中该用户没有给推荐的电影打分，就认为是打了0分。

In [14]:
def ndcg_benchmark(ratings_train, ratings_test, rated_movie_limit=10):
    """
    for each user_id
    1. get recommend list using ratings_train rated movies
    2. use ratings_test rated movies to validate ndcg value
    if it is not rated, make it zero
    return average ndcg_score for all ratings_test users
    """
    
    ndcg_score, count = (0, 0)
    #for user_id in np.sort(ratings_test['user_id'].unique()):
    for user_id in np.arange(1, 100):
        r = []
        #print('user_id %s' % user_id)
        highest_rate_movie_id = \
            ratings_train[(ratings_train.user_id == user_id) & \
                (ratings_train.rating == np.max(ratings_train[ratings_train.user_id == user_id].rating))][['movie_id']]
        
        highest_rate_movie_id_list = highest_rate_movie_id.movie_id.values
        # Performance will have significant impact if not limit the number of
        # rated movies for filtering 
        random.shuffle(highest_rate_movie_id_list)
        highest_rate_movie_id_list = highest_rate_movie_id_list[:rated_movie_limit]
        
        recommend_list = genre_recommend_by_movid_list(highest_rate_movie_id_list)
        #print(user_id, recommend_list)
        
        for item in recommend_list:
            x = ratings_test[ratings_test.user_id == user_id][['movie_id', 'rating']]
            if x[x.movie_id == item].empty:
                r.append(0)
            else:
                r.append(\
                    np.asscalar(ratings_test[(ratings_test.user_id == user_id) & \
                        (ratings_test.movie_id == item)].rating.values))
    
        ndcg_score += ndcg_at_k(r, len(r))
        count += 1.0

    ndcg_score /= count
    return ndcg_score

print(ndcg_benchmark(ratings_train, ratings_test))

0.15149997257459272


可以看到hit ratio和NDCG的值都偏低。原因包括该验证集并不是由新的推荐算法的产生的，实际生产中更多会通过A/B方式做验证。